In [266]:
!pip install selenium

In [267]:
import numpy as np
import pandas as pd
import requests 
from pandasql import sqldf
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
import time
import re

In [413]:
driver = webdriver.Chrome("C:/Users/manis/Desktop/chromedriver/chromedriver.exe")
url = "https://jindal.utdallas.edu/som/internship-stories/#loadNextStories"
driver.get(url)
wait = WebDriverWait(driver, 10)

degrees = ["Business Analytics", "Accounting Flex", "Finance", 
           "Information Technology and Management", "Management Science", 
           "Marketing", "Supply Chain Management"]

href = []
for degree in degrees:
    input_degree = driver.find_element_by_xpath('//*[@id="semester-flexdatalist"]')
    input_degree.send_keys(degree)
    
    time.sleep(2)
    try:
        search_button = wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="semester-flexdatalist-results"]/li/span[1]/span')))
        search_button.click()
    except (NoSuchElementException, TimeoutException):
        time.sleep(3)
        search_button = wait.until(ec.element_to_be_clickable((By.XPATH, '//*[@id="semester-flexdatalist-results"]/li/span[1]/span')))
        search_button.click()

    time.sleep(2)
    educ_level = driver.find_element_by_xpath('//*[@id="all-level"]/div[1]/input').click()
    
    time.sleep(2)
    for i in range(1, 8):
        n = 0
        time.sleep(1)
        reset = driver.find_element_by_xpath('//*[@id="semester"]').click()
        time.sleep(1)
        term = driver.find_element_by_xpath('//*[@id="all-semester"]/div[' + str(i) + ']/input').click()
        time.sleep(3)
        int_num = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div/div/div[2]/div[1]/div[1]/p').text
        time.sleep(2)
        int_num = int(int_num)
        int_num
        
        if int_num == 0:
            print("No internships in the following term")
            print("Degree : {} \nTerm : {} \nNumber of Internships : {}".format(degree, i, int_num))
            continue

        for j in range(1, int_num + 1):
            try:
                time.sleep(1)
                href_path = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div/div/div[2]/div[3]/div[' + str(j) + ']/div[2]/a').get_attribute("href")
                href.append(href_path)
                n = n + 1
            except NoSuchElementException:
                break

        print("Degree : {} \nTerm : {} \nNumber of Internships : {} \nRecorded Number of Internships : {}".format(degree, i, int_num, n))
        
    driver.back()

driver.close()

In [142]:
df_href = pd.DataFrame(href)

In [257]:
df_href.head()

,story_links
0,https://jindal.utdallas.edu/internship-stories...
1,https://jindal.utdallas.edu/internship-stories...
2,https://jindal.utdallas.edu/internship-stories...
3,https://jindal.utdallas.edu/internship-stories...
4,https://jindal.utdallas.edu/internship-stories...


In [562]:
df_href.shape

(1282, 1)

In [145]:
df_href.to_excel("C:/Users/manis/Desktop/Scraper/df_href.xlsx")

In [414]:
df = pd.DataFrame(columns = ["Student", "Degree", "Company", "Term", "Courses_Helped", "Source", "Link"])

for i in df_href["story_links"]:
    driver = webdriver.Chrome("C:/Users/manis/Desktop/chromedriver/chromedriver.exe")
    url = str(i)
    driver.get(url)
    wait = WebDriverWait(driver, 10)
    
    time.sleep(2) 
    heading = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[1]/div/h1').text
    try:
        description, term = heading.split(" - ")
    except ValueError:
        try:
            description, invalid, term = heading.split(" - ")
        except ValueError:
            try:
                description, invalid1, invalid2, term = heading.split(" - ")
            except ValueError:
                try:
                    description, invalid1, invalid2, invalid3, term = heading.split(" - ")
                except ValueError:
                    continue
                
    degree = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[1]').text
    company = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[2]').text
    course_header = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/h3[3]').text
    
    if course_header == "Courses Helped":
        try:
            courses_helped = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[3]').text
            source_header = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/h3[5]').text
            if source_header == "What helped you to get this Internship?":
                try:
                    source =  driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]').text
                    student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[6]').text
                except NoSuchElementException:
                    source =  driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]/text()').text
                    student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[6]').text
            else:
                source = np.nan
                student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]').text
        except NoSuchElementException:
            courses_helped = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[3]/text()').text
            source_header = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/h3[5]').text
            if source_header == "What helped you to get this Internship?":
                try:
                    source =  driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]').text
                    student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[6]').text
                except NoSuchElementException:
                    source =  driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]/text()').text
                    student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[6]').text
            else:
                source = np.nan
                student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]').text
    else:
        courses_helped = np.nan
        source_header = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/h3[4]').text
        if source_header == "What helped you to get this Internship?":
            try:
                source =  driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[4]').text
                student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]').text
            except NoSuchElementException:
                source =  driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[4]/text()').text
                student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[5]').text
        else:
            source = np.nan
            student = driver.find_element_by_xpath('//*[@id="mainhtml"]/body/div[2]/div[2]/p[4]').text
    
    df = df.append({"Student" : student,
                    "Degree" : degree,
                    "Company" : company,
                    "Term" : term,
                    "Courses_Helped" : courses_helped,
                    "Source" : source,
                    "Link" : url}, ignore_index=True)

    driver.close()

In [293]:
df.head()

,Unnamed: 0,Student,Degree,Company,Term,Courses_Helped,Source,Link
0,0,Chakradhar Reddy Kothakapa,Business Analytics,American National Bank of Texas,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,"Internet (e.g. company website, job search eng...",https://jindal.utdallas.edu/internship-stories...
1,1,Jaypriya Ubriani,Business Analytics,"Kanarys, Inc.",Fall 2018,NaN,"Internet (e.g. company website, job search eng...",https://jindal.utdallas.edu/internship-stories...
2,2,Huizhen Chen,Business Analytics,East Capital Funding,Fall 2018,"MIS 6380: DATA VISUALIZATION, OPRE 6302: OPERA...",JSOM Handshake,https://jindal.utdallas.edu/internship-stories...
3,3,Di Lei,Business Analytics,Alpha Business Images,Fall 2018,MIS 6344: WEB ANALYTICS,"Internet (e.g. company website, job search eng...",https://jindal.utdallas.edu/internship-stories...
4,4,Rahul Singh,Business Analytics,Adobe Systems,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,"Internet (e.g. company website, job search eng...",https://jindal.utdallas.edu/internship-stories...


In [294]:
print("The shape of the dataframe : {}".format(df.shape))

The shape of the dataframe : (1282, 8)


In [194]:
df.to_excel("C:/Users/manis/Desktop/Scraper/data.xlsx")

In [550]:
df = pd.read_excel("data.xlsx")

In [551]:
df[df.columns] = df[df.columns].astype(str)

In [552]:
def RemoveBrackets(string):
    clean = re.sub(r" ?\([^)]+\)", "", string)
    return clean

In [553]:
df["Source"] = df["Source"].apply(RemoveBrackets)

In [554]:
df.head()

,Student,Degree,Company,Term,Courses_Helped,Source,Link
0,Chakradhar Reddy Kothakapa,Business Analytics,American National Bank of Texas,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,"Internet, Personal connection",https://jindal.utdallas.edu/internship-stories...
1,Jaypriya Ubriani,Business Analytics,"Kanarys, Inc.",Fall 2018,nan,"Internet, Personal connection",https://jindal.utdallas.edu/internship-stories...
2,Huizhen Chen,Business Analytics,East Capital Funding,Fall 2018,"MIS 6380: DATA VISUALIZATION, OPRE 6302: OPERA...",JSOM Handshake,https://jindal.utdallas.edu/internship-stories...
3,Di Lei,Business Analytics,Alpha Business Images,Fall 2018,MIS 6344: WEB ANALYTICS,Internet,https://jindal.utdallas.edu/internship-stories...
4,Rahul Singh,Business Analytics,Adobe Systems,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,"Internet, Personal connection, Professional De...",https://jindal.utdallas.edu/internship-stories...


In [555]:
df["Degree"].value_counts()

Business Analytics                            396
Information Technology and Management         393
Supply Chain Management                       223
Accounting Flex                               122
Management Science                             88
Supply Chain Management, MBA                   23
Business Analytics, MBA                        18
Information Technology and Management, MBA     12
Management Science, MBA                         7
Name: Degree, dtype: int64

In [556]:
def ITM(string):
    if string == "Information Technology and Management":
        degrees = "IT and Management"
    elif string == "Information Technology and Management, MBA":
        degrees = "IT and Management, MBA"
    else:
        degrees = string
    
    return degrees

In [557]:
df["Degree"] = df["Degree"].apply(ITM)

In [558]:
df["Degree"].value_counts()

Business Analytics              396
IT and Management               393
Supply Chain Management         223
Accounting Flex                 122
Management Science               88
Supply Chain Management, MBA     23
Business Analytics, MBA          18
IT and Management, MBA           12
Management Science, MBA           7
Name: Degree, dtype: int64

In [559]:
df_new = df[df["Term"] != "Spring 2018"]
df_new = df[df["Term"] != "Summer 2018"]

In [560]:
df_new["Term"].value_counts()

Spring 2019    325
Summer 2019    317
Fall 2019      317
Fall 2018      301
Spring 2020     15
Spring 2018      6
Name: Term, dtype: int64

In [561]:
df_new.to_excel("C:/Users/manis/Desktop/Scraper/new_data.xlsx")

In [531]:
split_courses = (df.set_index(df.columns.drop('Courses_Helped',1).tolist())
                        .Courses_Helped.str.split(', ', expand=True)
                        .stack()
                        .reset_index()
                        .rename(columns={0:'Courses_Helped'})
                        .loc[:, df.columns])

In [532]:
split_courses.head()

,Student,Degree,Company,Term,Courses_Helped,Source,Link
0,Chakradhar Reddy Kothakapa,Business Analytics,American National Bank of Texas,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,"Internet, Personal connection",https://jindal.utdallas.edu/internship-stories...
1,Jaypriya Ubriani,Business Analytics,"Kanarys, Inc.",Fall 2018,nan,"Internet, Personal connection",https://jindal.utdallas.edu/internship-stories...
2,Huizhen Chen,Business Analytics,East Capital Funding,Fall 2018,MIS 6380: DATA VISUALIZATION,JSOM Handshake,https://jindal.utdallas.edu/internship-stories...
3,Huizhen Chen,Business Analytics,East Capital Funding,Fall 2018,OPRE 6302: OPERATIONS MANAGEMENT,JSOM Handshake,https://jindal.utdallas.edu/internship-stories...
4,Di Lei,Business Analytics,Alpha Business Images,Fall 2018,MIS 6344: WEB ANALYTICS,Internet,https://jindal.utdallas.edu/internship-stories...


In [533]:
split_courses["Courses_Helped"] = split_courses["Courses_Helped"].replace("nan", np.nan)
split_courses["Source"] = split_courses["Source"].replace("nan", np.nan)

In [534]:
split_courses = split_courses.dropna()

In [535]:
split_courses["Term"].value_counts()

Fall 2019      753
Summer 2019    699
Spring 2019    638
Fall 2018      584
Spring 2020     41
Spring 2018      8
Summer 2018      1
Name: Term, dtype: int64

In [536]:
split_courses = split_courses[split_courses["Term"] != "Spring 2018"]
split_courses = split_courses[split_courses["Term"] != "Summer 2018"]

In [537]:
split_courses.head()

,Student,Degree,Company,Term,Courses_Helped,Source,Link
0,Chakradhar Reddy Kothakapa,Business Analytics,American National Bank of Texas,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,"Internet, Personal connection",https://jindal.utdallas.edu/internship-stories...
2,Huizhen Chen,Business Analytics,East Capital Funding,Fall 2018,MIS 6380: DATA VISUALIZATION,JSOM Handshake,https://jindal.utdallas.edu/internship-stories...
3,Huizhen Chen,Business Analytics,East Capital Funding,Fall 2018,OPRE 6302: OPERATIONS MANAGEMENT,JSOM Handshake,https://jindal.utdallas.edu/internship-stories...
4,Di Lei,Business Analytics,Alpha Business Images,Fall 2018,MIS 6344: WEB ANALYTICS,Internet,https://jindal.utdallas.edu/internship-stories...
5,Rahul Singh,Business Analytics,Adobe Systems,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,"Internet, Personal connection, Professional De...",https://jindal.utdallas.edu/internship-stories...


In [538]:
split_courses.to_excel("C:/Users/manis/Desktop/Scraper/courses.xlsx")

In [539]:
split_source = (df.set_index(df.columns.drop('Source',1).tolist())
                        .Source.str.split(', ', expand=True)
                        .stack()
                        .reset_index()
                        .rename(columns={0:'Source'})
                        .loc[:, df.columns])

In [540]:
split_source["Courses_Helped"] = split_source["Courses_Helped"].replace("nan", np.nan)
split_source["Source"] = split_source["Source"].replace("nan", np.nan)

In [541]:
split_source = split_source.dropna()

In [542]:
def Other(string):
    if string not in ["Internet", "Personal connection", "JSOM Handshake", "JSOM career event", "JSOM on-campus interview", "Faculty referral"]:
        source = "Other"
    else:
        source = string
    
    return source

In [543]:
split_source["Source"] = split_source["Source"].apply(Other)

In [544]:
split_source.head()

,Student,Degree,Company,Term,Courses_Helped,Source,Link
0,Chakradhar Reddy Kothakapa,Business Analytics,American National Bank of Texas,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,Internet,https://jindal.utdallas.edu/internship-stories...
1,Chakradhar Reddy Kothakapa,Business Analytics,American National Bank of Texas,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,Personal connection,https://jindal.utdallas.edu/internship-stories...
4,Huizhen Chen,Business Analytics,East Capital Funding,Fall 2018,"MIS 6380: DATA VISUALIZATION, OPRE 6302: OPERA...",JSOM Handshake,https://jindal.utdallas.edu/internship-stories...
5,Di Lei,Business Analytics,Alpha Business Images,Fall 2018,MIS 6344: WEB ANALYTICS,Internet,https://jindal.utdallas.edu/internship-stories...
6,Rahul Singh,Business Analytics,Adobe Systems,Fall 2018,BUAN 6337: MKT PREDICTIVE ANALYTICS SAS,Internet,https://jindal.utdallas.edu/internship-stories...


In [545]:
split_source["Source"].value_counts()

Internet                    685
Personal connection         416
JSOM Handshake              322
JSOM career event           175
Other                        77
JSOM on-campus interview     63
Faculty referral             62
Name: Source, dtype: int64

In [546]:
split_source = split_source[split_source["Term"] != "Spring 2018"]
split_source = split_source[split_source["Term"] != "Summer 2018"]

In [547]:
split_source["Term"].value_counts()

Fall 2019      516
Summer 2019    436
Spring 2019    428
Fall 2018      397
Spring 2020     17
Name: Term, dtype: int64

In [548]:
split_source.to_excel("C:/Users/manis/Desktop/Scraper/sources.xlsx")